In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import os
import keras.backend as K
from elmo.lm_generator import LMDataGenerator, MTLLMDataGenerator
from elmo.model_girnet import ELMo

Using TensorFlow backend.


In [3]:
# import fastBPE
# bpe = fastBPE.fastBPE("../twiter_scrapping/preprocess/fastBPE/codes", "../twiter_scrapping/data/vocab.en.es.40000")

## Note
- fasttext: sent_vector: unsup: [0.5040783036202435, 0.44986425566245447]
- fasttext: sent_vector: sup: [0.5628058728541675, 0.5187983749157842]
- fasttext: lstm: unsup [0.6182707995419401, 0.5768815131210775] 
- fasttext: lstm: sup [0.6705, 0.6624]
- MultiBPEmb: lstm: unsup: 0.52
- MultiBPEmb: lstm: sup: 0.64

## Word Vocab

### cm_p_test_1000.txt
- vanilla :  46.75967671612614
- mtl: 39.42567465448776
- girnet: 165.677334

###  cm_main_test_1000.txt
- vanilla : 53.722(4), 73.619(32)
- mtl: 58.723(32), 46.54(3)
- girnet: 22(4), 115(64)

## BPE
###  cm_main_test_1000.txt
- vanilla_bpe: 51.959(1), 53.7539(4), 91.7316(64)
- mlt_bpe: 46.614(4)
- girnet_bpe: 18.187604(4)

Conclusion: S4UBWORDS RULE!

In [3]:
# DATA_SET_DIR = '../twiter_scrapping/data/'
DATA_SET_DIR = '../../data/Dataset_VACS/Language Model'

In [4]:
# parameters = {
#     'name': 'cm_mtl_4',
#     'multi_processing': True,
#     'n_threads': 10,
#     'cuDNN': True if len(K.tensorflow_backend._get_available_gpus()) else False,
#     'train_dataset': 'cm_p_train_small.txt',
#     'valid_dataset': 'cm_p_test_1000.txt',
#     'test_dataset': 'cm_test_main_1000.txt',
#     'vocab': 'vocab',
#     'vocab_size': 525133,
#     'num_sampled': 8000,
#     'charset_size': 262,
#     'sentence_maxlen': 32,
#     'token_maxlen': 50,
#     'token_encoding': 'word',
#     'epochs': 1,
#     'patience': 2,
#     'batch_size': 4,
#     'clip_value': 1,
#     'cell_clip': 5,
#     'proj_clip': 5,
#     'lr': 0.2,
#     'shuffle': True,
#     'n_lstm_layers': 1,
#     'n_highway_layers': 1,
#     'cnn_filters': [[1, 32],
#                     [2, 32],
#                     [3, 64],
#                     [4, 128],
#                     [5, 256],
#                     [6, 512],
#                     [7, 512]
#                     ],
#     'lstm_units_size': 400,
#     'hidden_units_size': 200,
#     'char_embedding_size': 16,
#     'dropout_rate': 0.1,
#     'word_dropout_rate': 0.05,
#     'weight_tying': True,
#     'unidirectional': True,
#     'subwords': False,
# }

# parameters = {
#     'name': 'cm_mtl_bpe_4',
#     'multi_processing': True,
#     'n_threads': 10,
#     'cuDNN': True if len(K.tensorflow_backend._get_available_gpus()) else False,
#     'train_dataset': 'cm_p_train_small.txt.40000',
#     'valid_dataset': 'cm_p_test_1000.txt.40000',
#     'test_dataset': 'cm_test_main_1000.txt.40000',
#     'vocab': 'vocab.en.es.40000.processed',
#     'vocab_size': 54497,
#     'num_sampled': 2000,
#     'charset_size': 262,
#     'sentence_maxlen': 32,
#     'token_maxlen': 50,
#     'token_encoding': 'word',
#     'epochs': 1,
#     'patience': 2,
#     'batch_size': 4,
#     'clip_value': 1,
#     'cell_clip': 5,
#     'proj_clip': 5,
#     'lr': 0.2,
#     'shuffle': True,
#     'n_lstm_layers': 1,
#     'n_highway_layers': 1,
#     'cnn_filters': [[1, 32],
#                     [2, 32],
#                     [3, 64],
#                     [4, 128],
#                     [5, 256],
#                     [6, 512],
#                     [7, 512]
#                     ],
#     'lstm_units_size': 400,
#     'hidden_units_size': 200,
#     'char_embedding_size': 16,
#     'dropout_rate': 0.1,
#     'word_dropout_rate': 0.05,
#     'weight_tying': True,
#     'unidirectional': True,
#     'subwords': False
# }


parameters = {
    'name': 'cm_girnet_vcs_data',
    'multi_processing': True,
    'n_threads': 10,
    'cuDNN': True if len(K.tensorflow_backend._get_available_gpus()) else False,
    'train_dataset': 'cm_p_train_small.txt',
    'valid_dataset': 'valid.txt',
    'test_dataset': 'test.txt',
    'vocab': 'fvocab',
    'vocab_size': 12688,
    'num_sampled': 2000,
    'charset_size': 262,
    'sentence_maxlen': 32,
    'token_maxlen': 50,
    'token_encoding': 'word',
    'epochs': 1,
    'patience': 2,
    'batch_size': 4,
    'clip_value': 1,
    'cell_clip': 5,
    'proj_clip': 5,
    'lr': 0.2,
    'shuffle': False,
    'n_lstm_layers': 1,
    'n_highway_layers': 1,
    'cnn_filters': [[1, 32],
                    [2, 32],
                    [3, 64],
                    [4, 128],
                    [5, 256],
                    [6, 512],
                    [7, 512]
                    ],
    'lstm_units_size': 200,
    'hidden_units_size': 100,
    'char_embedding_size': 16,
    'dropout_rate': 0.1,
    'word_dropout_rate': 0.05,
    'weight_tying': True,
    'unidirectional': True,
    'subwords': False,
}

In [7]:
train_generator = MTLLMDataGenerator([os.path.join(DATA_SET_DIR, 'english.txt'),
                                      os.path.join(DATA_SET_DIR, 'hindi.txt'),
                                      os.path.join(DATA_SET_DIR, 'cm.txt')],
                                  os.path.join(DATA_SET_DIR, parameters['vocab']),
                                  sentence_maxlen=parameters['sentence_maxlen'],
                                  token_maxlen=parameters['token_maxlen'],
                                  batch_size=parameters['batch_size'],
                                  shuffle=parameters['shuffle'],
                                  token_encoding=parameters['token_encoding']
)
val_generator = MTLLMDataGenerator([os.path.join(DATA_SET_DIR, 'valid.txt'),
                                    os.path.join(DATA_SET_DIR, 'valid.txt'),
                                    os.path.join(DATA_SET_DIR, 'valid.txt')],
                                  os.path.join(DATA_SET_DIR, parameters['vocab']),
                                  sentence_maxlen=parameters['sentence_maxlen'],
                                  token_maxlen=parameters['token_maxlen'],
                                  batch_size=parameters['batch_size'],
                                  shuffle=parameters['shuffle'],
                                  token_encoding=parameters['token_encoding']
)
test_generator = MTLLMDataGenerator([os.path.join(DATA_SET_DIR, parameters['test_dataset']),os.path.join(DATA_SET_DIR, parameters['test_dataset']),os.path.join(DATA_SET_DIR, parameters['test_dataset'])],
                                  os.path.join(DATA_SET_DIR, parameters['vocab']),
                                  sentence_maxlen=parameters['sentence_maxlen'],
                                  token_maxlen=parameters['token_maxlen'],
                                  batch_size=parameters['batch_size'],
                                  shuffle=parameters['shuffle'],
                                  token_encoding=parameters['token_encoding']
)

In [6]:
train_en_generator =  LMDataGenerator(os.path.join(DATA_SET_DIR, 'en_p_train_small.txt'),
                                  os.path.join(DATA_SET_DIR, parameters['vocab']),
                                  sentence_maxlen=parameters['sentence_maxlen'],
                                  token_maxlen=parameters['token_maxlen'],
                                  batch_size=parameters['batch_size'],
                                  shuffle=parameters['shuffle'],
                                  token_encoding=parameters['token_encoding'])

train_es_generator =  LMDataGenerator(os.path.join(DATA_SET_DIR, 'es_p_train_small.txt'),
                                  os.path.join(DATA_SET_DIR, parameters['vocab']),
                                  sentence_maxlen=parameters['sentence_maxlen'],
                                  token_maxlen=parameters['token_maxlen'],
                                  batch_size=parameters['batch_size'],
                                  shuffle=parameters['shuffle'],
                                  token_encoding=parameters['token_encoding'])

train_cm_generator =  LMDataGenerator(os.path.join(DATA_SET_DIR, 'cm_p_train_small.txt'),
                                  os.path.join(DATA_SET_DIR, parameters['vocab']),
                                  sentence_maxlen=parameters['sentence_maxlen'],
                                  token_maxlen=parameters['token_maxlen'],
                                  batch_size=parameters['batch_size'],
                                  shuffle=parameters['shuffle'],
                                  token_encoding=parameters['token_encoding'])

test_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['test_dataset']),
                                os.path.join(DATA_SET_DIR, parameters['vocab']),
                                sentence_maxlen=parameters['sentence_maxlen'],
                                token_maxlen=parameters['token_maxlen'],
                                batch_size=parameters['batch_size'],
                                shuffle=parameters['shuffle'],
                                token_encoding=parameters['token_encoding'])
val_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['valid_dataset']),
                                os.path.join(DATA_SET_DIR, parameters['vocab']),
                                sentence_maxlen=parameters['sentence_maxlen'],
                                token_maxlen=parameters['token_maxlen'],
                                batch_size=parameters['batch_size'],
                                shuffle=parameters['shuffle'],
                                token_encoding=parameters['token_encoding'])

In [6]:
# Set-up Generators
train_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['train_dataset']),
                                  os.path.join(DATA_SET_DIR, parameters['vocab']),
                                  sentence_maxlen=parameters['sentence_maxlen'],
                                  token_maxlen=parameters['token_maxlen'],
                                  batch_size=parameters['batch_size'],
                                  shuffle=parameters['shuffle'],
                                  token_encoding=parameters['token_encoding'])

val_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['valid_dataset']),
                                os.path.join(DATA_SET_DIR, parameters['vocab']),
                                sentence_maxlen=parameters['sentence_maxlen'],
                                token_maxlen=parameters['token_maxlen'],
                                batch_size=parameters['batch_size'],
                                shuffle=parameters['shuffle'],
                                token_encoding=parameters['token_encoding'])

test_generator = LMDataGenerator(os.path.join(DATA_SET_DIR, parameters['test_dataset']),
                                os.path.join(DATA_SET_DIR, parameters['vocab']),
                                sentence_maxlen=parameters['sentence_maxlen'],
                                token_maxlen=parameters['token_maxlen'],
                                batch_size=parameters['batch_size'],
                                shuffle=parameters['shuffle'],
                                token_encoding=parameters['token_encoding'])

In [8]:
# Compile ELMo
elmo_model = ELMo(parameters)
elmo_model.compile_elmo(print_summary=False)

In [10]:
# for tr in [train_en_generator, train_es_generator, train_cm_generator]:
#     elmo_model.train(train_data=tr, valid_data=val_generator)
# 4.6
elmo_model.train(train_data=train_generator, valid_data=val_generator)

Epoch 1/1
313/313 [==============================] - 119s 379ms/step - loss: 3.1002 - val_loss: 6.5039

Epoch 00001: val_loss improved from inf to 6.50392, saving model to /app/vendors/ELMo-keras/data/models/elmo_best_weights.hdf5
Training took 118.8310923576355 sec


In [9]:
elmo_model.load_temp()

In [14]:
elmo_model.save(sampled_softmax=False, temp=False)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Language Model saved successfully


In [ ]:
! echo "trained" > status.txt

In [15]:
# cm_p_test_1000.txt
# vanilla :  46.75967671612614
# mtl: 39.42567465448776
# girnet: 165.677334
# cm_main_test_10010.txt
# vanilla :  73.61951568054178
# vanilla_4 : 53.722699251300426
# mtl: 58.723475872121696
# mtl_4:
# girnet: 115.844843
# girnet_4: 22.249534

# vanilla_bpe: 91.73161858208685
# vanilla_bpe_4: 53.753941728994036
# vanilla bpe_1: 51.95905007072605
# mlt bpe_4: 46.614687442883785
# girnet_bpe_4: 18.187604

elmo_model.evaluate(test_generator)

Langauge Model Perplexity 1: 752.939329


KeyboardInterrupt: 

In [17]:
elmo_model._model.evaluate_generator(train_generator)

In [ ]:
# Build ELMo meta-model to deploy for production and persist in disk
# elmo_model.wrap_multi_elmo_encoder(print_summary=True, save=True)

In [ ]:
# elmo_model.load()

In [ ]:
# Get ELMo embeddings to feed as inputs for downstream tasks
# elmo_embeddings = elmo_model.get_outputs(test_generator, output_type='word', state='mean')